### Data cleaning(tab name: Primary care acess equity)

**Purpose** : The values in the Age Group and State fields appear to be mixed. Before begining the analysis, organize the data type and clean it. 

### 1. Load the data

In [65]:
import pandas as pd
import numpy as np
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [66]:
# load the data
pc = pd.read_csv("Primary care access equity.csv")
pc.head()

,client_id,AgeGroup,State,Visit Count with PCP,InsuranceStatus
0,00585C81758229D353F1957B647EE60E,OH,18–21,2,Insured
1,00599B3D3CEE691CB64104CFE0084057,22–35,OH,2,Insured
2,00D20AD70A0BFEC75C9B9EA7A446AB89,18–21,OH,2,Insured
3,012824D10C0C4787AF2F77FE3250D831,OH,18–21,2,Insured
4,013317A7CA515BD67917E8C0BB74F08D,OH,22–35,3,Insured


**As above, some values of Agegroup and State look swapped**

### 2.1 Check Whitespace(empty space)

In [68]:
# Checking count of values having empty spaces in each cell in AgeGroup, State columns
cols = ['AgeGroup', 'State']

for c in cols:
    print(f'\n Column: {c}')
    print(
        pc.loc[
            pc[c].astype(str).str.contains(r'\s', regex=True),
            c
        ]
        .map(lambda x: f'[{x}]')
        .value_counts()
    )


 Column: AgeGroup
AgeGroup
[Over 45]     164
[Under 18]     91
Name: count, dtype: int64

 Column: State
State
[Over 45]     300
[Under 18]    201
[OH ]          20
Name: count, dtype: int64


Both the AgeGroup and State columns contain the value **“OH”** in each row. However, empty spaces were found only in the **State** column. 
Therefore, the empty spaces associated with “OH” in the State column needed to be removed.

In [69]:
# check count of value in State column
pc['State'].value_counts()

State
OH          530
22–35       380
Over 45     300
Under 18    201
36–45       158
18–21       121
OH           20
PA            1
Name: count, dtype: int64

As above, **two 'OH's** are listed in the State Column, which means one of the OHs might have empty spaces..

### 2.2 Remove whitespaces

In [70]:
# Leading and trailing empty spaces in each string is removed
# Ex: [OH ] -> [OH]
pc['State'] = (
    pc['State']
    .astype(str)
    .str.replace(r'[\u00A0\t\r\n]+', ' ', regex=True) 
    .str.replace(r'\s+', ' ', regex=True)             
    .str.strip()                                       
)

In [71]:
pc['State'].value_counts()

State
OH          550
22–35       380
Over 45     300
Under 18    201
36–45       158
18–21       121
PA            1
Name: count, dtype: int64

As a result, empty spaces in OH are removed

 ### 3.1 Swap State and AgeGroup values

In [72]:
# Swap State and AgeGroup values when State contains a valid U.S. state abbreviation

US_STATES = {
    'AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA',
    'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD',
    'MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
    'NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
    'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','DC'
}

def swap_if_state_not_valid(row):
    # Extract values from the row
    state = row['State']
    age   = row['AgeGroup']

    # Clean the State value (allow NaN)
    # - Strip leading/trailing whitespace
    # - Convert to uppercase for consistent comparison
    state_clean = str(state).strip().upper() if pd.notna(state) else None

    # If the State value is a valid U.S. state abbreviation,
    # swap the values between State and AgeGroup
    if state_clean in US_STATES:
        return row

    # Otherwise, do not perform a swap
    row['State'], row['AgeGroup'] = age, state
    return row

In [73]:
pc = pc.apply(swap_if_state_not_valid, axis=1)

### 3.2 Result

In [74]:
print(pc)

                             client_id AgeGroup State  Visit Count with PCP  \
0     00585C81758229D353F1957B647EE60E    18–21    OH                     2   
1     00599B3D3CEE691CB64104CFE0084057    22–35    OH                     2   
2     00D20AD70A0BFEC75C9B9EA7A446AB89    18–21    OH                     2   
3     012824D10C0C4787AF2F77FE3250D831    18–21    OH                     2   
4     013317A7CA515BD67917E8C0BB74F08D    22–35    OH                     3   
...                                ...      ...   ...                   ...   
1706  FF55DFB59948058D23FE28DDD6492440    22–35   NaN                     6   
1707  FF55DFB59948058D23FE28DDD6492440    22–35    OH                     6   
1708  FF650295A520431FCF18CD73C529CE59    22–35    OH                    10   
1709  FF90F5317B13A90AEBEA5B4598F64759    18–21    OH                     1   
1710  FFD3313CC7BC8C7E4F79E49A275930CE    18–21    OH                     1   

     InsuranceStatus  
0            Insured  
1    

In [75]:
pc['State'].value_counts()

State
OH    1696
IL       1
NJ       1
FL       1
PA       1
AZ       1
Name: count, dtype: int64

In State Column, only values of valid U.S. state abbreviations exist.

In [77]:
pc['AgeGroup'].value_counts()

AgeGroup
22–35       543
Over 45     464
Under 18    292
36–45       218
18–21       194
Name: count, dtype: int64

In State AgeGroup, only values related age exist.

**The values in the State and AgeGroup columns were swapped correctly according to the specified conditions.**

### 4. Remove null values 

In [80]:
# Checking for missing values(null value)
p_null=pc.isnull().sum()
p_null[p_null>0]

State    10
dtype: int64

**Number of Null value in State: 10**

In [81]:
#Remove null values
pc = pc.dropna(subset=['State'])

In [82]:
# Vaildate the result
pc['State'].isna().sum()

np.int64(0)

As a result, null values are removed.

### 5. Export the cleaned data

In [83]:
# excel
pc.to_excel('cleaned_Primary care access equity.xlsx', index=False)

# csv
pc.to_csv('cleaned_Primary care access equity.csv', index=False)